### 用語

入力値：\\( x \\)

重みベクトル：\\( w \\)

総入力：\\( z = w_1x_1 + ... + w_mx_m = w^Tx \\)

しきい値：\\( \theta \\)


活性化関数(単位ステップ関数、ヘビサイド関数):\\( \phi \\)

$$ \phi = \begin{eqnarray}\left\{\begin{array}{l}1(z\geqq\theta) \\ -1(z<\theta)\end{array}\right.\end{eqnarray} $$

学習率：\\( \eta \\)

### パーセプトロンの学習規則

1. 重みを0または値の小さい乱数で初期化
1. トレーニングサンプル\\( x^{(i)} \\)ごとに以下の手順を実行

a.出力値\\( \hat{y} \\)を計算する

b.重みを更新する

#### 各重み\\( w_j \\)の更新法
$$ w_j := w_j + \Delta w_j $$

#### \\( \Delta w_j \\)の算出
$$ \Delta w_j = \eta(y^{(i)} - \hat{y}^{(i)})x_j^{(i)} $$
\\( y^{(i)} \\)：本当のクラスラベル

\\( \hat{y}^{(i)} \\)：予測されたクラスラベル

#### 予測が当たった場合は重みは更新されない

$$ \Delta w_j = \eta(1 - 1)x_j^{(i)} = 0 $$
$$ \Delta w_j = \eta(-1 - (-1))x_j^{(i)} = 0 $$

#### 予測が当たらなかった場合は重みが更新される

$$ \Delta w_j = \eta(-1 - 1)x_j^{(i)} = \eta(-2)x_j^{(i)} $$
$$ \Delta w_j = \eta(1 - (-1))x_j^{(i)} = \eta(2)x_j^{(i)} $$

#### 　\\( w_0 \\) のみ違う計算式となる

$$ \Delta w_0 = \eta(-1 - output^{(i)}) $$
$$ \Delta w_1 = \eta(1 - output^{(i)})x_1^{(i)} $$

In [4]:
import numpy as np

class Perceptron(object):
    """
    eta: 学習率(0.0<eta<1.0)
    n_iter: トレーニング回数
    """
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
        
    def fit(self, X, y):
        """
        X: トレーニングデータ（行にサンプル、列に特徴量）
        Y: 目的変数
        """
        self.w_ = np.zeros(1 + X.shape[1])
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                #重みw1,...,wmの更新
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update + xi
                # 重みw0の更新
                self.w_[0] += update
                # 重みの更新が0でない場合は誤分類としてカウント
                errors += int(update != 0.0)
            # 反復回数ごとの誤差を格納
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        """総入力を計算"""
        return np.dot(X, self.w_[1:]) + self.w_[0]
    
    def predict(self, X):
        """1ステップ後のクラスラベルを返す"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)
                
            

In [2]:
#  重み更新
print('重み更新あり', int (0.5 != 0.0))
print('重み更新なし', int (0.0 != 0.0))

重み更新あり 1
重み更新なし 0


In [3]:
import numpy as np

data = np.array([100, 200, 300, 400, 500])
# 値を取得
print(data[data < 300])
# インデックスを取得
print(np.where(data < 300))
# 真の場合、偽の場合で1, -1を返す
print(np.where(data < 300, 1, -1))

[100 200]
(array([0, 1], dtype=int64),)
[ 1  1 -1 -1 -1]
